In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from imblearn.over_sampling import SMOTE
import datetime, sys, sklearn

ROOT_DIR = Path(__file__).resolve().parents[1] if "__file__" in locals() else Path.cwd().parents[0]
RAW_PATH = ROOT_DIR / "data" / "raw" / "IBMTelco_Datensatz.csv"
OUT_DIR = ROOT_DIR / "data" / "processed"
OUT_DIR.mkdir(parents=True, exist_ok=True)
random_state = 42

In [2]:
df = pd.read_csv(RAW_PATH)
df = df.rename(columns=lambda c: c.strip())
df = df.drop(columns=["customerID"])
df["Churn"] = df["Churn"].map({"Yes":1, "No":0}).astype(int)

In [3]:
from sklearn.model_selection import StratifiedShuffleSplit

y = df["Churn"]
sss1 = StratifiedShuffleSplit(n_splits=1, test_size=0.15, random_state=random_state)
train_val_idx, test_idx = next(sss1.split(df, y))
train_val = df.iloc[train_val_idx].reset_index(drop=True)
test = df.iloc[test_idx].reset_index(drop=True)

sss2 = StratifiedShuffleSplit(n_splits=1, test_size=0.1764705882, random_state=random_state)
train_idx, val_idx = next(sss2.split(train_val, train_val["Churn"]))
train = train_val.iloc[train_idx].reset_index(drop=True)
val = train_val.iloc[val_idx].reset_index(drop=True)

In [4]:
df_list = [train, val, test]
for d in df_list:
    d["TotalCharges"] = d["TotalCharges"].replace(" ", np.nan).astype(float)
    mask = d["tenure"] == 0
    d.loc[mask, "TotalCharges"] = d.loc[mask, "TotalCharges"].fillna(0)
    d["SeniorCitizen"] = d["SeniorCitizen"].astype(int)

In [5]:
# Ordinal mappings
mapping_contract = {"Month-to-month":1, "One year":12, "Two year":24}
mapping_internet = {"No":0, "DSL":1, "Fiber optic":2}
for d in df_list:
    d["Contract"] = d["Contract"].map(mapping_contract)
    d["InternetService"] = d["InternetService"].map(mapping_internet)

# One-Hot
onehot_cols = [
    'gender','Partner','Dependents','PhoneService','MultipleLines',
    'OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport',
    'StreamingTV','StreamingMovies','PaperlessBilling','PaymentMethod'
]
encoder = OneHotEncoder(sparse_output=False, drop="first", handle_unknown="ignore")

train_encoded = pd.DataFrame(encoder.fit_transform(train[onehot_cols]),
                             columns=encoder.get_feature_names_out(onehot_cols), index=train.index)
val_encoded = pd.DataFrame(encoder.transform(val[onehot_cols]),
                           columns=encoder.get_feature_names_out(onehot_cols), index=val.index)
test_encoded = pd.DataFrame(encoder.transform(test[onehot_cols]),
                            columns=encoder.get_feature_names_out(onehot_cols), index=test.index)

train = pd.concat([train.drop(columns=onehot_cols), train_encoded], axis=1)
val = pd.concat([val.drop(columns=onehot_cols), val_encoded], axis=1)
test = pd.concat([test.drop(columns=onehot_cols), test_encoded], axis=1)

for d in [train,val,test]:
    d.columns = d.columns.str.replace(" ", "_").str.replace("-", "_")

In [6]:
scaler = MinMaxScaler()
num_cols = ["tenure","MonthlyCharges","TotalCharges"]

train[num_cols] = scaler.fit_transform(train[num_cols])
val[num_cols] = scaler.transform(val[num_cols])
test[num_cols] = scaler.transform(test[num_cols])

In [7]:
from imblearn.over_sampling import SMOTE
from collections import Counter

X_train = train.drop(columns=["Churn"])
y_train = train["Churn"]

print("Before SMOTE:", Counter(y_train))
smote = SMOTE(random_state=random_state)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
print("After SMOTE:", Counter(y_train_res))

train_bal = pd.concat([X_train_res, y_train_res], axis=1)

Before SMOTE: Counter({0: 3621, 1: 1308})
After SMOTE: Counter({0: 3621, 1: 3621})


In [8]:
train_bal.to_csv(OUT_DIR / "train.csv", index=False)
val.to_csv(OUT_DIR / "val.csv", index=False)
test.to_csv(OUT_DIR / "test.csv", index=False)

In [9]:
all_cols = sorted(set(train.columns) | set(val.columns) | set(test.columns))

for name, df in zip(["train", "val", "test"], [train, val, test]):
    missing_cols = set(all_cols) - set(df.columns)
    for col in missing_cols:
        df[col] = 0
    df = df[all_cols]
    df.to_csv(ROOT_DIR / "data" / "processed" / f"{name}.csv", index=False)

In [10]:
now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

md = f"""# Data Preparation Summary

**Datum/Zeit:** {now}  
**random_state:** {random_state}  

---

## Overview
All preprocessing steps (Data Selection, Cleaning, Encoding, Scaling, SMOTE) were executed in one reproducible pipeline.  
Intermediate results were stored in `/data/processed/`.

---

## Processed Files
- data/processed/train.csv  
- data/processed/val.csv  
- data/processed/test.csv  

---

## Transformation Overview
| Phase | Transformation | Tools |
|--------|----------------|--------|
| 1 | Stratified Split (70/15/15) | scikit-learn StratifiedShuffleSplit |
| 2 | Cleaning (fix TotalCharges, types) | pandas |
| 3 | Encoding (ordinal + one-hot) | sklearn.preprocessing.OneHotEncoder |
| 4 | Scaling (0–1) | sklearn.preprocessing.MinMaxScaler |
| 5 | Balancing (SMOTE) | imblearn.over_sampling.SMOTE |

---

## Reproducibility
- Python {sys.version.split()[0]}  
- pandas {pd.__version__}  
- scikit-learn {sklearn.__version__}

---

## Note
This notebook ensures full reproducibility of the Data Preparation phase according to **CRISP-ML(Q)** standards.  
All subsequent modeling and deployment steps should use these processed datasets as input.
"""

REPORT_DIR = ROOT_DIR / "reports" / "data_preparation"
REPORT_DIR.mkdir(parents=True, exist_ok=True)
Path(REPORT_DIR / "data_preparation_summary.md").write_text(md, encoding="utf-8")

1118